In [1]:
%matplotlib inline
import pylab as plt
import numpy as np
from astrometry.util.fits import *
from collections import Counter
from astrometry.libkd.spherematch import match_radec
from astrometry.util.starutil_numpy import *
import fitsio
from glob import glob
import sys
import os
import pickle

In [7]:
H,W = 20, 20
ry = [2, 6, 14, 16]
k = 0
sky = 1
starflux = 3
rx = 10

In [20]:
maxrow = max(ry)
minrow = min(ry)
shift_all = maxrow + 1
skip = maxrow - np.array(ry)
after_rows = H - minrow

In [4]:
! mkdir -p guider-plots

In [33]:
def doplot(title):
    global k
    plt.figure(figsize=(12,4))
    for i,im in enumerate(ims):
        plt.subplot(1,4, i+1)
        plt.imshow(im, interpolation='nearest', origin='lower', vmin=0, vmax=120)
        #plt.colorbar()
    plt.suptitle(title)
    plt.savefig('guider-plots/plot%03i.png' % k)
    plt.close()
    k += 1

In [34]:
def do_exposure():
    for i in range(10):
        # Exposure
        exptime = 1
        for i,im in enumerate(ims):
            im[:,:] += sky * exptime
            im[ry[i],rx] += starflux * exptime
        doplot('exposure')

In [35]:
def do_shift_all():
    # shift_all
    for step in range(shift_all):
        exptime = 1
        for i,im in enumerate(ims):
            if step <= skip[i]:
                pass
            else:
                # shift image down one row
                im[:-1,:] = im[1:,:]
                im[-1,:] = 0.
            # expose
            im[:,:] += sky * exptime
            im[ry[i],rx] += starflux * exptime
        doplot('shift_all')

In [36]:
def do_readout():
    # readout
    for step in range(W):
        exptime = 1
        for i,im in enumerate(ims):
            # serial shift
            im[0,1:] = im[0,:-1]
            im[0,0] = 0
            # expose
            im[:,:] += sky * exptime
            im[ry[i],rx] += starflux * exptime
        doplot('readout')

In [37]:
def do_after_rows():
    # after_rows
    print('after_rows', after_rows)
    for step in range(after_rows):
        exptime = 1
        for i,im in enumerate(ims):
            # shift image down one row
            im[:-1,:] = im[1:,:]
            im[-1,:] = 0.
            # expose
            im[:,:] += sky * exptime
            im[ry[i],rx] += starflux * exptime
        doplot('after_rows')

In [41]:
k = 0
ims = [np.zeros([H,W]) for y in ry]

In [42]:
do_exposure()
do_shift_all()
do_readout()
do_after_rows()
do_exposure()
do_shift_all()
do_readout()
do_after_rows()
do_exposure()
do_shift_all()
do_readout()
do_after_rows()
do_exposure()
do_shift_all()
do_readout()
do_after_rows()

after_rows 18
after_rows 18
after_rows 18
after_rows 18


In [43]:
k

260

In [ ]:
! convert guider-plots/plot* anim.mov